In [ ]:
#SQL-script для создания таблиц
# create table if not exists customer(
#   customer_id int primary key,
#   first_name text,
#   last_name text,
#   gender text,
#   DOB date,
#   job_title text,
#   job_industry_category text,
#   wealth_segment text,
#   deceased_indicator text,
#   owns_car text,
#   address text,
#   postcode int,
#   state text,
#   country text,
#   property_valuation int
# );

# create table if not exists product(
#   product_id int,
#   brand text,
#   product_line text,
#   product_class text,
#   product_size text,
#   list_price decimal,
#   standard_cost decimal
# );

# create table if not exists product_cor as
# select * from 
# (select *,row_number() over(partition by product_id order by list_price desc) as rn from product)
# where rn = 1;
# ALTER TABLE product_cor ADD PRIMARY KEY (product_id);

# create table if not exists orders(
#   order_id int primary key,
#   customer_id int,
#   order_date date,
#   online_order bool,
#   order_status text
# );

# create table if not exists order_items(
#   order_item_id int primary key,
#   order_id int,
#   product_id int,
#   quantity int,
#   item_list_price_at_sale decimal,
#   item_standard_cost_at_sale decimal
# );

## Таблица customer
![alt text](img/1.png)

## Таблица product_cor
![alt text](img/2.png)

## Таблица orders
![alt text](img/3.png)

## Таблица order_items
![alt text](img/4.png)

In [ ]:
# Вывести все уникальные бренды, у которых есть хотя бы один продукт со стандартной стоимостью выше 1500 долларов, 
# и суммарными продажами не менее 1000 единиц.
# select distinct pc.brand
# from product_cor pc
# inner join order_items oi on pc.product_id = oi.product_id
# where pc.standard_cost > 1500
# group by pc.brand, pc.product_id
# having sum(oi.quantity) >= 1000;

![alt text](img/5.png)

In [ ]:
# Для каждого дня в диапазоне с 2017-04-01 по 2017-04-09 включительно вывести количество подтвержденных онлайн-заказов 
# и количество уникальных клиентов, совершивших эти заказы.
# select count(order_status) as confirmed_orders_count, count(distinct customer_id) as unique_customers_count
# from orders
# where order_date between '2017-04-01' and '2017-04-09' and order_status = 'Approved' and online_order = true
# group by order_date;

![alt text](img/6.png)

In [ ]:
# Вывести профессии клиентов:
# из сферы IT, чья профессия начинается с Senior;
# из сферы Financial Services, чья профессия начинается с Lead.
# Для обеих групп учитывать только клиентов старше 35 лет. Объединить выборки с помощью UNION ALL.
# select job_title
# from customer
# where age(current_date, dob) > '35 years'::interval and job_industry_category = 'IT' and job_title LIKE 'Senior%'
# union all
# select job_title
# from customer
# where age(current_date, dob) > '35 years'::interval and job_industry_category = 'Financial Services' and job_title LIKE 'Lead%';

![alt text](img/7.png)

In [ ]:
# Вывести бренды, которые были куплены клиентами из сферы Financial Services, но не были куплены клиентами из сферы IT.
# select distinct pc.brand
# from product_cor pc 
# inner join order_items oi on pc.product_id = oi.product_id
# inner join orders o on oi.order_id = o.order_id 
# inner join customer c on o.customer_id = c.customer_id 
# where c.job_industry_category = 'Financial Services' and pc.brand not in 
# (select distinct pc2.brand
# from product_cor pc2 
# inner join order_items oi2 on pc2.product_id = oi2.product_id
# inner join orders o2 on oi2.order_id = o2.order_id 
# inner join customer c2 on o2.customer_id = c2.customer_id 
# where c2.job_industry_category = 'IT');

![alt text](img/8.png)

In [ ]:
# Вывести 10 клиентов (ID, имя, фамилия), которые совершили наибольшее количество онлайн-заказов (в штуках) брендов Giant Bicycles, 
# Norco Bicycles, Trek Bicycles, при условии, что они активны и имеют оценку имущества (property_valuation) выше среднего среди клиентов из того же штата.
# select c.customer_id, c.first_name, c.last_name
# from customer c 
# inner join orders o on c.customer_id = o.customer_id
# inner join order_items oi on o.order_id = oi.order_id
# inner join product_cor pc on oi.product_id = pc.product_id
# where online_order = true and pc.brand in ('Giant Bicycles', 'Norco Bicycles', 'Trek Bicycles') 
# and c.deceased_indicator = 'N' and c.property_valuation > (select avg(property_valuation) from customer where state = c.state)
# group by c.customer_id, c.first_name, c.last_name
# order by count(o.order_id) desc
# limit 10;

![alt text](img/9.png)

In [ ]:
# Вывести всех клиентов (ID, имя, фамилия), у которых нет подтвержденных онлайн-заказов за последний год, 
# но при этом они владеют автомобилем и их сегмент благосостояния не Mass Customer.
# select c.customer_id, c.first_name, c.last_name
# from customer c
# left join orders o on c.customer_id = o.customer_id and o.online_order = 'Yes' and o.order_status = 'Approved'
# and o.order_date >= current_date - '1 year'::interval 
# WHERE o.order_id is null and c.owns_car = 'Yes' and c.wealth_segment != 'Mass Customer';

![alt text](img/10.png)

In [ ]:
# Вывести всех клиентов из сферы 'IT' (ID, имя, фамилия), которые купили 2 из 5 продуктов с самой высокой list_price в продуктовой линейке Road.
# with top_products as 
# (select product_id
# from product_cor
# where product_line = 'Road'
# order by list_price desc
# limit 5)
# select c.customer_id, c.first_name, c.last_name
# from customer c
# inner join orders o on c.customer_id = o.customer_id
# inner join order_items oi on o.order_id = oi.order_id
# inner join top_products tp on oi.product_id = tp.product_id
# WHERE c.job_industry_category = 'IT'
# group by c.customer_id, c.first_name, c.last_name
# having count(distinct oi.product_id) >= 2;

![alt text](img/11.png)

In [ ]:
# Вывести клиентов (ID, имя, фамилия, сфера деятельности) из сфер IT или Health, 
# которые совершили не менее 3 подтвержденных заказов в период 2017-01-01 по 2017-03-01, 
# и при этом их общий доход от этих заказов превышает 10 000 долларов.
# Разделить вывод на две группы (IT и Health) с помощью UNION.
# select c.customer_id, c.first_name, c.last_name, c.job_industry_category
# from customer c
# inner join orders o on c.customer_id = o.customer_id
# inner join order_items oi on o.order_id = oi.order_id
# WHERE c.job_industry_category = 'IT' and o.order_status = 'Approved' and o.order_date between '2017-01-01' and '2017-03-01'
# group by c.customer_id, c.first_name, c.last_name, c.job_industry_category
# having count(o.order_id) >= 3 and sum(oi.item_list_price_at_sale * oi.quantity) > 10000
# union
# select c.customer_id, c.first_name, c.last_name, c.job_industry_category
# from customer c
# inner join orders o on c.customer_id = o.customer_id
# inner join order_items oi on o.order_id = oi.order_id
# WHERE c.job_industry_category = 'Health' and o.order_status = 'Approved' and o.order_date between '2017-01-01' and '2017-03-01'
# group by c.customer_id, c.first_name, c.last_name, c.job_industry_category
# having count(o.order_id) >= 3 and sum(oi.item_list_price_at_sale * oi.quantity) > 10000;

![alt text](img/12.png)